### 1. Загрузим нужные библиотеки и напишем функции

In [25]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from tqdm.notebook import tqdm


def get_soup(url: str):
    
    res = requests.get(url)
    return BeautifulSoup(res.content, 'html.parser')


def get_cars_hrefs(tree):
    cars = tree.find_all('a', {'class': 'vehicle-card-link'})
    return [
        {'model': car.h2.text, 'href': car.get('href')} for car in cars
    ]


def columns_maker(table):
    soup = BeautifulSoup(str(table), 'html.parser')

    dt = soup.find_all('dt')
    
    columns = []
    for i in range(len(dt)):
        columns.append(dt[i].text)
    
    #columns.append('Price')

    return columns


def table_maker(table):
    soup = BeautifulSoup(str(table), 'html.parser')

    dd = soup.find_all('dd')

    res = []
    for i in range(len(dd)):
        res.append(dd[i].text.strip())
    
    #new_line_ind = res[3].find('\n')
    #if new_line_ind != -1:
    #    res[3] = res[3][:new_line_ind]
    #
    return res


MAIN_URL = 'https://www.cars.com/'

### 2. Соберем нужное количество данных

In [6]:
data = []
for p in tqdm(range(50, 101)):
    url = MAIN_URL + f'shopping/results/?list_price_max=&makes[]=&maximum_distance=all&models[]=&page={p}&stock_type=used&zip='
    tree = get_soup(url)
    hlp = get_cars_hrefs(tree)
    data.extend(hlp)
    sleep(0.5)

  0%|          | 0/51 [00:00<?, ?it/s]

In [7]:
for p in tqdm(range(101, 151)):
    url = MAIN_URL + f'shopping/results/?list_price_max=&makes[]=&maximum_distance=all&models[]=&page={p}&stock_type=used&zip='
    tree = get_soup(url)
    hlp = get_cars_hrefs(tree)
    data.extend(hlp)
    sleep(0.5)

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
for p in tqdm(range(151, 201)):
    url = MAIN_URL + f'shopping/results/?list_price_max=&makes[]=&maximum_distance=all&models[]=&page={p}&stock_type=used&zip='
    tree = get_soup(url)
    hlp = get_cars_hrefs(tree)
    data.extend(hlp)
    sleep(0.5)

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
for p in tqdm(range(201, 251)):
    url = MAIN_URL + f'shopping/results/?list_price_max=&makes[]=&maximum_distance=all&models[]=&page={p}&stock_type=used&zip='
    tree = get_soup(url)
    hlp = get_cars_hrefs(tree)
    data.extend(hlp)
    sleep(0.5)

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
cars = pd.DataFrame(data)
cars

,model,href
0,2021 Kia K5 GT-Line,/vehicledetail/76b6012f-7391-4dd2-abd9-58ab328...
1,2019 Lincoln MKT Reserve,/vehicledetail/0f6ae29c-cbdf-4388-a488-c90c37d...
2,2019 Mercedes-Benz GLS 450 Base 4MATIC,/vehicledetail/f48e124a-a217-4d4c-8ee7-5035540...
3,2016 Acura RLX Technology Package,/vehicledetail/d98e846e-832a-414d-9aae-286275f...
4,2022 Toyota Sequoia Limited,/vehicledetail/7111cdbb-af1c-4305-b99a-a9a185b...
...,...,...
4211,2019 Mercedes-Benz SLC 300 Base,/vehicledetail/3c4f3f41-f974-4124-b8a6-ba98797...
4212,2019 Honda Accord Sport,/vehicledetail/7cb65bae-b08d-4b29-8a51-2d8f514...
4213,2019 Nissan Armada Platinum,/vehicledetail/f2dd0af8-91f7-48ce-beb4-3c69e28...
4214,2015 RAM ProMaster City Tradesman,/vehicledetail/9c8007bd-de98-4289-8f3d-b65d0cf...


---

In [90]:
url = MAIN_URL + cars['href'][0]
tree = get_soup(url)

basics = tree.find('dl', {'class': 'fancy-description-list'})
history = tree.find('section', {'class': 'sds-page-section vehicle-history-section'})

In [91]:
final_columns = columns_maker(basics)
final_columns.extend(columns_maker(history)[:-1])
final_columns.append('Price')
final_columns

['Exterior color',
 'Interior color',
 'Drivetrain',
 'MPG',
 'Fuel type',
 'Transmission',
 'Engine',
 'VIN',
 'Stock #',
 'Mileage',
 'Accidents or damage',
 '1-owner vehicle',
 'Personal use only',
 'Price']

In [100]:
cars_df = pd.DataFrame(columns=final_columns)

In [110]:
curr_cars = cars.iloc[393:1001, :]
for href in tqdm(curr_cars['href']):
    url = MAIN_URL + href
    tree = get_soup(url)
    
    basics = tree.find('dl', {'class': 'fancy-description-list'})
    history = tree.find('section', {'class': 'sds-page-section vehicle-history-section'})
    
    if basics != None and history != None:    
        price = tree.find('span', {'class': 'primary-price'}).text
        res = table_maker(basics)
        res.extend(table_maker(history.dl))
        
        
        if len(res) == 14:
            res = res[:-1]
        
        if len(res) == 13:
            res.append(price)
            cars_df.loc[href] = res
    
    sleep(0.5)

  0%|          | 0/608 [00:00<?, ?it/s]

In [113]:
curr_cars = cars.iloc[1001:2001, :]
for href in tqdm(curr_cars['href']):
    url = MAIN_URL + href
    tree = get_soup(url)
    
    basics = tree.find('dl', {'class': 'fancy-description-list'})
    history = tree.find('section', {'class': 'sds-page-section vehicle-history-section'})
    
    if basics != None and history != None:    
        price = tree.find('span', {'class': 'primary-price'}).text
        res = table_maker(basics)
        res.extend(table_maker(history.dl))
        
        
        if len(res) == 14:
            res = res[:-1]
        
        if len(res) == 13:
            res.append(price)
            cars_df.loc[href] = res
    
    sleep(0.5)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [115]:
curr_cars = cars.iloc[2001:3001, :]
for href in tqdm(curr_cars['href']):
    url = MAIN_URL + href
    tree = get_soup(url)
    
    basics = tree.find('dl', {'class': 'fancy-description-list'})
    history = tree.find('section', {'class': 'sds-page-section vehicle-history-section'})
    
    if basics != None and history != None:    
        price = tree.find('span', {'class': 'primary-price'}).text
        res = table_maker(basics)
        res.extend(table_maker(history.dl))
        
        
        if len(res) == 14:
            res = res[:-1]
        
        if len(res) == 13:
            res.append(price)
            cars_df.loc[href] = res
    
    sleep(0.5)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [118]:
curr_cars = cars.iloc[3042:, :]
for href in tqdm(curr_cars['href']):
    url = MAIN_URL + href
    tree = get_soup(url)
    
    basics = tree.find('dl', {'class': 'fancy-description-list'})
    history = tree.find('section', {'class': 'sds-page-section vehicle-history-section'})
    
    if basics != None and history != None:    
        price = tree.find('span', {'class': 'primary-price'}).text
        res = table_maker(basics)
        res.extend(table_maker(history.dl))
        
        
        if len(res) == 14:
            res = res[:-1]
        
        if len(res) == 13:
            res.append(price)
            cars_df.loc[href] = res
    
    sleep(0.5)

  0%|          | 0/1174 [00:00<?, ?it/s]

In [128]:
cars_df.drop_duplicates(inplace=True)

### 3. Сохраним таблицу в файл

In [141]:
cars_df.to_csv('cars_df.csv', index=False)